In [8]:
import os 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import requests
import cv2 
from numpy.polynomial import Polynomial
from skimage.morphology import skeletonize, binary_closing, square, binary_erosion, binary_dilation, medial_axis, thin, disk, convex_hull_image, diamond, star, isotropic_closing, remove_small_objects
from skimage.filters import threshold_otsu
import json
import gpxpy
import plotly.graph_objects as go

In [11]:
# took frame every 3 seconds to provide semantic road segmentation model a frame
time_interval = 3

vid_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/videos/2023_Ronde_van_Vlaanderen_1.mp4"

stitched_imgs_dir_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/main_3_stitched_frames"
segm_imgs_dir_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/main_3_segmentation_masks"
heli_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/csv-files/2023 Ronde van Vlaanderen 1 heli.csv"
gpx_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/gpx-files/2023 Ronde van Vlaanderen course.gpx"

visualization = True
mapbox_access_token = "pk.eyJ1Ijoic2VubmVsb29idXljayIsImEiOiJjbG1zeDVreHowZmw5Mm1xY2lxeTFtYTB2In0.ymj-xZAfeeivbksi1PJttw"

In [3]:
def read_csv(path) :
    df = pd.read_csv(path)
    
    df['lat'] = df.apply(lambda row: float(row['lat']) * 1 if row['NS'] == 'N' else float(row['lat']) * (-1), axis=1)
    df['lng'] = df.apply(lambda row: float(row['lng']) * 1 if row['EW'] == 'E' else float(row['lng']) * (-1), axis=1)
    
    # make new df with only lat, lng, alt and seconds_from_start
    df = df[['lat', 'lng', 'alt', 'seconds_from_start']]

    return df

In [6]:
def read_gpx(path) : 
    # read in course data
    # Parse GPX file
    gpx_file = open(gpx_path, 'r')
    gpx = gpxpy.parse(gpx_file)

    # extract latitude and longitude coordinates for each point
    route = [(point.latitude, point.longitude) for track in gpx.tracks for segment in track.segments for point in segment.points]
    route_df = pd.DataFrame(route, columns=['latitude', 'longitude'])
    return route_df

In [7]:
def get_intersection_coordinates(lat, lon, radius) : 
    df_intersections = pd.DataFrame(columns=['lat', 'lon'])

    overpass_url = "http://overpass-api.de/api/interpreter"


    # first get the ways around the point 
    # .w is a temporary variable that stores the result of the first query
    # node(way_cnt:2-);   // select those nodes that are members of two or more of the given ways
    overpass_query = f"""[out:json];
                        way(around: 300, {lat},{lon})["highway"]->.w;
                        node(way_cnt.w:2-);  
                        out;"""

    response = requests.get(overpass_url, params={'data': overpass_query})

    for item in response.json()['elements']:
        df_intersections.loc[len(df_intersections)] = {'lat': item['lat'], 'lon': item['lon']}


    # print(df_intersections)
    return df_intersections

In [ ]:
def get_overview_map(heli_df, gpx_df, seconds, mb_token) : 
    mapbox_route = go.Scattermapbox(
        lon=gpx_df['longitude'],
        lat=gpx_df['latitude'],
        name="Route points",
        mode='markers',
        marker=dict(
            size=8,
            color='red',
        ),
    )

    mapbox_heli_other = go.Scattermapbox(
        lon=heli_df['lng'],
        lat=heli_df['lat'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
        ),
        name="Other Helicopter points"
    )

    if seconds in heli_df['seconds_from_start'].values:
        filtered_df = heli_df[heli_df['seconds_from_start'] == seconds]

    filtered_df_first = filtered_df.iloc[0]

    intersec_df = get_intersection_coordinates(filtered_df_first['lat'], filtered_df_first['lng'])

    mapbox_heli_now = go.Scattermapbox(
        lon=filtered_df['lng'],
        lat=filtered_df['lat'],
        mode='markers',
        marker=dict(
            color='yellow',
            size=8,
        ),
        name="Live Helicopter point(s)"
    )

    mapbox_intersec= go.Scattermapbox(
        lon=intersec_df['lon'],
        lat=intersec_df['lat'],
        # mode = 'markers',
        mode='markers',
        marker=dict(
            color='black',
            size=8,
        ),
        name="Intersections",
        textposition="top right",
        textfont=dict(
            color='black',
            size=8,
        )
    )

    # now set up the layout of the map
    layout = go.Layout(
        width=1200,
        height=900,
        mapbox=dict(
            accesstoken=mb_token,
            style="open-street-map",
            zoom=16,
            center=dict(
                lon=filtered_df.iloc[0]['lng'],
                lat=filtered_df.iloc[0]['lat']
            )
        )
    )

    data = [mapbox_route, mapbox_heli_other, mapbox_heli_now, mapbox_intersec]

    fig = go.Figure(data=data, layout=layout)

    

    







    

In [ ]:
# read in video
cap = cv2.VideoCapture(vid_path)

# read in heli csv
df_heli = read_csv(heli_path)

# read in gpx
df_gpx = read_gpx(gpx_path)

# get fps 
fps = cap.get(cv2.CAP_PROP_FPS)

# set time interval (in seconds) for frame capture
interval = time_interval

# calculate frame interval based on fps and time interval
# frame interval in #frames
frame_interval = int(fps * interval)

# frame counter 
frame_cnt = 0

if visualization : 
    # loop over frames in video
    while cap.isOpened():
        # read next frame from video
        ret, frame = cap.read()

        # check if frame was read successfully
        if not ret:
            break

        seconds = frame_cnt / fps

        if visualization :
            overview_map_img = get_overview_map(df_heli, df_gpx, seconds, mapbox_access_token)

        # check if it's time to capture a frame
        if frame_cnt % frame_interval == 0:
            continue


        frame_cnt += 1 
    